# Lotka Volterra UPINN

In [1]:
import torch
import sys

# Set the seed for reproducibility
torch.manual_seed(42)

# Add the parent directory of the script (i.e., project/) to sys.path
sys.path.append('../')

from utils.BVP import BVP
from utils.UPINN import UPINN
from utils.DataGenerators import LotkaVolterra, sample_with_noise
from utils.Architectures import FNN
from utils.Plotters import LV_Plotter

Consider the Lotka-Volterra equations, which describe the dynamics of a predator-prey system:

\begin{align}
\frac{dx}{dt} &= \alpha x - \beta x y, \\
\frac{dy}{dt} &= - \delta y + \gamma x y,
\end{align}

where $x$ is the number of prey, $y$ is the number of predators, and $\alpha$, $\beta$, $\gamma$, and $\delta$ are positive constants.

## Generate Data from System

In [2]:
###############################################
### Generate data from Lotka-Volterra model ###
###############################################
###   dx/dt = alpha*x - beta*x*y            ###
###   dy/dt = gamma*x*y - delta*y           ###
###############################################
alpha, beta, gamma, delta = 1.3, 0.9, 0.8, 1.8
x0, y0 = 0.44249296, 4.6280594

# alpha, beta, gamma, delta = 2/3, 4/3, 1.0, 1.0
# x0, y0 = 1.0, 1.0

LV = LotkaVolterra(alpha, beta, gamma, delta, torch.tensor([x0, y0], dtype=torch.float32))

time_int = [0, 3]
# time_int = [0, 25]
train_test = 1.0
# train_test = 0.8
N = 800
t = torch.linspace(time_int[0], time_int[1], N)
X = LV.solve(t)
train_idx = torch.arange(0, train_test*N, dtype=torch.long)
test_idx = torch.arange(train_test*N, N, dtype=torch.long)

# Sample subset and add noise
t_d, X_d = sample_with_noise(30, t[train_idx], X[train_idx], epsilon=1e-1)

### Setup Boundary Value Problem

Assume that $\gamma x y$ is not known, and we want to learn it from data.

In [3]:
class LV_BVP(BVP):
    
    def __init__(self, params, X0):
        super().__init__(params)
        self.X0 = X0

    def f(self, z, U):
        alpha = self.params['alpha'] if 'alpha' in self.params else self.alpha
        beta = self.params['beta'] if 'beta' in self.params else self.beta
        delta = self.params['delta'] if 'delta' in self.params else self.delta
        gamma = self.params['gamma'] if 'gamma' in self.params else self.gamma

        dUdt = torch.cat([
        torch.autograd.grad(outputs=U[:, i], inputs=z, grad_outputs=torch.ones_like(U[:, i]), create_graph=True)[0]
        for i in range(U.shape[1])
        ], dim=-1)

        return torch.stack([
            dUdt[:, 0] - alpha*U[:, 0], # + beta*U[:, 0]*U[:, 1],
            dUdt[:, 1] + delta*U[:, 1] # - gamma*U[:, 0]*U[:, 1] # <-- Estimate this
        ], dim=-1)

    def g(self, z, U):
        return U - self.X0

# Define model
# params = dict(
#     alpha=torch.nn.Parameter(torch.tensor(0.5)),
#     beta=torch.nn.Parameter(torch.tensor(0.5)),
#     delta=torch.nn.Parameter(torch.tensor(0.5))
# )

params = dict(
    alpha=torch.nn.Parameter(torch.tensor(1.0)),
    beta=torch.nn.Parameter(torch.tensor(1.0)),
    delta=torch.nn.Parameter(torch.tensor(1.0)),
    # delta=delta,
    gamma=torch.nn.Parameter(torch.tensor(1.0))
)

params = dict(
    alpha=alpha,
    beta=beta,
    delta=delta,
    gamma=gamma
)

bvp = LV_BVP(params, torch.tensor([x0, y0], dtype=torch.float32))

### Setup UPINN

In [4]:
# Define model architectures
# hidden = [16] * 4
# u = FNN(
#     dims=[1, *hidden, 2],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.ReLU(),
# )
# G = FNN(
#     dims=[2, *hidden, 2],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.ReLU(),
# )


u = FNN(
    dims=[1, *[64, 64], 2],
    hidden_act=torch.nn.Sigmoid(),
    output_act=torch.nn.Identity(),
)
G = FNN(
    dims=[2, *[16, 16], 2],
    hidden_act=torch.nn.Sigmoid(),
    output_act=torch.nn.Identity(),
)

# Setup scaling layer
u.scale_fn = lambda t_: (t_-t.min())/(t.max()-t.min())
mu, sigma = 0, 2
epsilon = 1e-8
G.scale_fn = lambda x: (x-mu)/(sigma+epsilon)


upinn = UPINN(u, G, bvp=bvp)

Make a class that can plot the results of the UPINN.

In [5]:
plotter = LV_Plotter(t, X, t_d, X_d, -beta*X[:, 0]*X[:, 1] , gamma*X[:, 0]*X[:, 1])

In [6]:
plots = plotter(upinn.u, upinn.G)
plots["Solution"].show()
plots["Missing Terms"].show()

## Train UPINN

In [7]:
upinn.train(
    data_points=t_d.unsqueeze(-1),
    data_target=X_d,
    boundary_points=torch.tensor([[0.0]]),
    collocation_points=t[train_idx].unsqueeze(-1).requires_grad_(True),
    epochs=10000,
    # log_wandb=dict(name='UPINN', project='Master-Thesis', plotter=plotter, plot_interval=1000),
    log_wandb=None,
    optimizer=torch.optim.AdamW,
    optimizer_args=dict(lr=1e-2, weight_decay=1e-10),
    beta_softadapt=0.1,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau,
    scheduler_args=dict(factor=0.5, patience=500, min_lr=1e-6),
    loss_tol=1e-6,
    # lambda_reg=1e-3,
    # priotize_pde=100.0
    )

Beginning training...
Running on: cuda


100%|██████████| 10000/10000 [01:30<00:00, 110.53it/s, Loss=0.00154, BC Loss=2e-6, PDE Loss=3.8e-5, Data Loss=0.00458, Reg Loss=0.263, LR_reg=0]     

Training complete.


In [8]:
plots = plotter(upinn.u, upinn.G)
plots["Solution"].show()
plots["Missing Terms"].show()

## Improve physics loss

In [9]:
# Extend collocation points to include the entire time interval and train again
upinn.train(
    data_points=t_d.unsqueeze(-1),
    data_target=X_d,
    boundary_points=torch.tensor([[0.0]]),
    collocation_points=t[train_idx].unsqueeze(-1).requires_grad_(True),
    epochs=10000,
    log_wandb=None,
    optimizer=torch.optim.AdamW,
    optimizer_args=dict(lr=3e-4, weight_decay=1e-10),
    beta_softadapt=0.1,
    loss_tol=1e-6,
    priotize_pde=1000,
    )

Beginning training...
Running on: cuda


100%|██████████| 10000/10000 [01:28<00:00, 112.94it/s, Loss=0.00176, BC Loss=1e-6, PDE Loss=0.000515, Data Loss=0.00477, Reg Loss=0.267, LR_reg=0] 


Training complete.


In [10]:
plots = plotter(upinn.u, upinn.G)
plots["Solution"].show()
plots["Missing Terms"].show()

## Train beyond the data

In [11]:
# Freeze G and parameters
G.requires_grad_(False);
bvp.requires_grad_(False);

# Extend collocation points to include the entire time interval and train again
upinn.train(
    data_points=t_d.unsqueeze(-1),
    data_target=X_d,
    boundary_points=torch.tensor([[0.0]]),
    collocation_points=t.unsqueeze(-1).requires_grad_(True),
    epochs=10000,
    log_wandb=None,
    optimizer=torch.optim.AdamW,
    optimizer_args=dict(lr=3e-4, weight_decay=1e-10),
    beta_softadapt=0.1,
    loss_tol=1e-6,
    priotize_pde=1000,
    )

Beginning training...
Running on: cuda


100%|██████████| 10000/10000 [01:26<00:00, 115.54it/s, Loss=0.00176, BC Loss=1e-6, PDE Loss=0.000506, Data Loss=0.00477, Reg Loss=0.267, LR_reg=0]


Training complete.


In [12]:
plots = plotter(upinn.u, upinn.G)
plots["Solution"].show()
plots["Missing Terms"].show()